In [36]:
import numpy as np
import numpy.linalg as la
import sympy as sp
import pandas as pd
import scipy.optimize as opt
import ols
import time as time

Given the data set, we are tasked with estimating the following regression model by maximum likelihood ($\alpha_{t} = \beta_{0,t}$):

\begin{equation}
ln(w_{i,t}) = \beta_{0,t} + \beta_{1,t} Educ_{i,t} + \beta_{2,t} Age_{i,t} + \beta_{3,t} Black_{i,t} + \beta_{4,t} Hispanic_{i,t} + \beta_{5,t} OtherRace_{i,t} + \epsilon_{i,t}
\end{equation}

To estimate the parameters $ \beta_t = \{\beta_{i,t}\}_{i=0}^{5}$ by maximum likelihood estimation, we begin by assuming that $\epsilon$ ~ $\text{Norm}(0,\sigma^2)$. This allows us to express the model in terms of the following components for $n_t$ observations:

\begin{align*}
\epsilon \sim & \ \frac{1}{\sqrt{2\pi \sigma^2}}e^{\frac{-(y_t - X_t\beta_t)^2}{2\sigma}} \\
L(\beta_t, \sigma_t) = & \ \Pi_{i=1}^{n} \Bigg[ \frac{1}{\sqrt{2\pi \sigma_t^2}}e^{\frac{-\left(y_t - X_t\beta \right)^2}{2\sigma_t^2}} \Bigg] = \frac{1}{(2\pi \sigma_t)^{\frac{n_t}{2}}}e^{\frac{-\left(y_t - X_t\beta \right)^T\left(y_t - X_t\beta_t\right)}{2\sigma_t^2}} \\
log \left( L(\beta,\sigma) \right) = & \ -\frac{n}{2}\log(2\pi) - \frac{n}{2}\log(\sigma^2) - \frac{1}{2\sigma^2}((y - Xb)' \cdot (y - Xb))
\end{align*}

In [51]:
# Clean data, convert final datasets to arrays

data = pd.read_stata("PS3_data.dta")
r_dummy = pd.get_dummies(data.hrace)
r_dummy = r_dummy.rename(columns = {1.0:"white", 2.0:"black", 3.0:"hispanic", 8.0:"other"})
r_dummy = r_dummy.drop(["other"], axis=1)
data = pd.concat([data, r_dummy], axis=1)
data["logwage"] = np.log(data.hlabinc / data.hannhrs)
data = data[data.hsex == 1.0]
data = data[data.age < 61]
data = data[data.age > 24]
data = data[np.exp(data.logwage) > 7]
data["dummy"] = data.logwage*0 + 1
data = data[["year", "logwage", "dummy", "hyrsed", "age", "black", "hispanic"]]
data = data.dropna(axis=0, how="any")

data71 = data[data.year == 1971]
data80 = data[data.year == 1980]
data90 = data[data.year == 1990]
data00 = data[data.year == 2000]

A71 = np.array(data71.drop(["year"], axis=1))
A80 = np.array(data80.drop(["year"], axis=1))
A90 = np.array(data90.drop(["year"], axis=1))
A00 = np.array(data00.drop(["year"], axis=1))

It should be noted for $hrace$, only the values of 1,2,3, and 8 are exhibited, corresponding to White, African-American, Hispanic, and Other [1]. As after cleaning out NaN's there were not any "other" individuals remaining for the years in question, and hence would result in a singular matrix to compute our estimators with, we elected to leave it out of our estimation. We then obtain the following final regression equation:

\begin{equation}
ln(w_{i,t}) = \beta_{0,t} + \beta_{1,t} Educ_{i,t} + \beta_{2,t} Age_{i,t} + \beta_{3,t} Black_{i,t} + \beta_{4,t} Hispanic_{i,t} + \epsilon_{i,t}
\end{equation}

In [48]:
# initialize our parameter guesses

b0 = 3
b1 = 1
b2 = 2
b3 = 1
b4 = 1
s  = 1
b_g = [s, b0, b1, b2 ,b3, b4]


# Create a function to evaluate the log likelihood

def log_like_func(params, mat):
    s,b0,b1,b2,b3,b4 = params
    b = np.array([b0,b1,b2,b3,b4])
    n = len(mat)
    y = mat[:,0]
    X = mat[:,1:]
    Xb= X @ b
    log_l = (-(n/2)*np.log(2*np.pi) - (n/2)*np.log(s**2) 
             - (1/(2*s**2)) * ((y - Xb).T @ (y - Xb)))
    return -log_l

In [52]:
# Now use a minimization routine to solve our problem

bnds = ((1e-5,None),(None,None),(None,None),(None,None),
        (None,None),(None,None))

start = time.time()
results_mle_71 = opt.minimize(log_like_func, b_g, args=(A71), bounds=bnds)
results_mle_80 = opt.minimize(log_like_func, b_g, args=(A80), bounds=bnds)
results_mle_90 = opt.minimize(log_like_func, b_g, args=(A90), bounds=bnds)
results_mle_00 = opt.minimize(log_like_func, b_g, args=(A00), bounds=bnds)
end = time.time()

print("1971 bhat vector (MLE):", results_mle_71.x[1:])
print("1971 Sigma squared est:", results_mle_71.x[0])
print("   ")
print("1980 bhat vector (MLE):", results_mle_80.x[1:])
print("1980 Sigma squared est:", results_mle_80.x[0])
print("   ")
print("1990 bhat vector (MLE):", results_mle_90.x[1:])
print("1990 Sigma squared est:", results_mle_90.x[0])
print("   ")
print("2000 bhat vector (MLE):", results_mle_00.x[1:])
print("2000 Sigma squared est:", results_mle_00.x[0])
print("   ")
print("Time to complete:", end-start,"seconds")
print("   ")
print("------------------------------------------------------------------------------")
print("   ")

y71 = A71[:,0]
X71 = A71[:,1:]
y80 = A80[:,0]
X80 = A80[:,1:]
y90 = A90[:,0]
X90 = A90[:,1:]
y00 = A00[:,0]
X00 = A00[:,1:]

Bbar_71 = la.inv(X71.T @ X71) @ X71.T @ y71
Bbar_80 = la.inv(X80.T @ X80) @ X80.T @ y80
Bbar_90 = la.inv(X90.T @ X90) @ X90.T @ y90
Bbar_00 = la.inv(X00.T @ X00) @ X00.T @ y00

print("1971 bhat vector (OLS):", Bbar_71)
print("   ")
print("1980 bhat vector (OLS):", Bbar_80)
print("   ")
print("1990 bhat vector (OLS):", Bbar_90)
print("   ")
print("2000 bhat vector (OLS):", Bbar_00)

1971 bhat vector (MLE): [ 1.5509741   0.0668783   0.01439133 -0.16388592  0.03068459]
1971 Sigma squared est: 0.410099858229
   
1980 bhat vector (MLE): [ 1.61311822  0.06755369  0.01269864 -0.10271192  0.01354949]
1980 Sigma squared est: 0.449243224746
   
1990 bhat vector (MLE): [ 1.11861596  0.09755586  0.01346557 -0.1720357  -0.05958184]
1990 Sigma squared est: 0.48360419907
   
2000 bhat vector (MLE): [ 1.16113062  0.10921085  0.01098068 -0.24571021 -0.06045022]
2000 Sigma squared est: 0.539630721261
   
Time to complete: 0.31919169425964355 seconds
   
------------------------------------------------------------------------------
   
1971 bhat vector (OLS): [ 1.55095983  0.06687871  0.01439143 -0.16388759  0.03068781]
   
1980 bhat vector (OLS): [ 1.61308527  0.06755602  0.01269848 -0.10273541  0.01351133]
   
1990 bhat vector (OLS): [ 1.11858153  0.09755777  0.01346556 -0.1720245  -0.05971238]
   
2000 bhat vector (OLS): [ 1.16112971  0.10921098  0.01098074 -0.24570975 -0.060446

Remarks: All MLE estimates terminated as expected, and are identical to the OLS estimates derived later to examine accuracy; this is exactly as expected. We would, however, expect $\sigma^2_{MLE}$ to be biased, whereas $\sigma^2_{OLD}$ would be unbiased. As we have a very large data set, this is not extremely relevant.

Question 4: As we proceed through the decades, $Educ$ remains small for 1971 and 1980 but abruptly rises for 1990 to 0.0976 and further still in 2000 to 0.1092. While a number of factors contributed to this increase, it is likely that the advent of the "information age" helped increase the need for intellectual capital. This phenomenon has also arguably helped contribute to the overall rise in income inequality since 1971.

Sources:

[1] ftp://ftp.isr.umich.edu/pub/src/psid/codebook/FAM1971_codebook.pdf (page 119)